In [1]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import torch
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.optuna import OptunaSearch
from torch import nn
import numpy as np
from torch import nn
from ray.util.client import ray


import sys
import torch
import pandas as pd

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
#from pytorch_lightning.callbacks import TQDMProgressBar
import numpy as np
import tempfile
from ray import tune
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback

from pytorch_lightning.callbacks import ModelCheckpoint
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.lightning import (
    LightningTrainer,
    LightningConfigBuilder,
    LightningCheckpoint,
)


from models.transformer.kw_transformer import TransAm
from kw_transformer_functions import RMSELoss
from models.transformer.my_functrions import make_dataset, get_torch_data_loaders



In [2]:

torch.manual_seed(0)
np.random.seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [3]:
df = pd.read_csv(
    '../datasets/prepared/eth/ta_corr_01_fd.csv',
    parse_dates=True
)
target_col = 'log_returns'
df = df.set_index(['timestamp'])
df.index = pd.to_datetime(df.index)
exclude_cols = ['log_close', 'Close']
df

,log_close,Close,log_returns,diff_fear_greed,volatility_kcp,volume_em,volume_vpt,momentum_stoch_rsi,volatility_bbp,momentum_stoch,...,trend_macd_diff,trend_stc,volume_sma_em,trend_aroon_ind,momentum_tsi,trend_aroon_down,trend_vortex_ind_neg,trend_psar_down_indicator,trend_adx_neg,volatility_bbli
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-19,6.946110,1039.099976,0.002718,-0.008588,0.102478,31.624126,1.400980e+08,0.037616,0.422813,39.600400,...,-36.805177,0.000000,-60.164463,56.0,21.811785,20.0,0.992330,0.0,26.633272,0.0
2018-01-20,7.051986,1155.150024,0.105875,0.094570,0.454029,176.900699,4.550768e+08,0.209534,0.564169,57.400634,...,-32.297077,0.000000,-48.724324,56.0,20.500950,16.0,0.955632,0.0,24.958464,0.0
2018-01-21,6.956145,1049.579956,-0.095840,-0.107145,0.189903,-69.272645,1.352358e+08,0.033644,0.361546,41.207862,...,-35.591281,0.000000,-63.621180,56.0,17.246075,12.0,0.996763,0.0,24.462554,0.0
2018-01-22,6.911010,1003.260010,-0.045135,-0.056441,0.117475,-326.973658,-4.768732e+08,0.000000,0.255003,34.103119,...,-39.740806,0.000000,-96.179267,56.0,13.818018,8.0,1.069065,0.0,27.014842,0.0
2018-01-23,6.893889,986.229004,-0.017121,-0.028427,0.169233,-109.817190,-2.285243e+08,0.000000,0.217811,31.490833,...,-42.249176,0.000000,-118.460975,56.0,10.820051,4.0,1.120754,0.0,26.135747,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31,7.535900,1874.130493,-0.014249,-1.014249,0.765721,-18.994183,-6.215059e+07,0.627877,0.807676,67.930616,...,11.160775,94.726423,4.602985,60.0,2.887780,28.0,0.886572,0.0,18.486077,0.0
2023-06-01,7.529515,1862.201416,-0.006385,1.993615,0.590639,-9.265335,-1.205696e+08,0.493067,0.701580,60.614706,...,8.930618,97.363212,4.538830,60.0,2.991828,24.0,0.908415,0.0,18.237269,0.0
2023-06-02,7.553421,1907.256592,0.023907,-2.976093,0.989787,13.536856,1.116329e+08,0.865498,0.953481,88.246317,...,9.968121,98.681606,5.120619,60.0,4.968846,20.0,0.869446,0.0,16.780865,0.0


In [8]:
checkpoint_path = 'checkpoints/multioutput/best_checkpoints/fac_dense_timestep=5epoch=44-val_loss=0.569.ckpt'
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

ntest=42

print(checkpoint["hyper_parameters"])

#loss_fn=checkpoint["hyper_parameters"]['loss_fn']
batch_size = checkpoint["hyper_parameters"]['batch_size']
#num_layers= checkpoint["hyper_parameters"]['num_layers']
dropout =  checkpoint["hyper_parameters"]['dropout']
#nhead= checkpoint["hyper_parameters"]['nhead']
#feature_size= checkpoint["hyper_parameters"]['feature_size']
learning_rate= checkpoint["hyper_parameters"]['learning_rate']
weight_decay=checkpoint["hyper_parameters"]['weight_decay']

timestep=checkpoint["hyper_parameters"]['timestep']
horizon=checkpoint["hyper_parameters"]['horizon']

attn_type = checkpoint["hyper_parameters"]['attn_type']
model_checkpoint_outputdir = 'multioutput/btc_eth'
patience = 30
n_epochs = 100

{'loss_fn': <function RMSELoss at 0x000002007BA070D0>, 'batch_size': 4, 'feature_size': 36, 'decoder_size': 16, 'timestep': 5, 'horizon': 14, 'num_layers': 4, 'dropout': 0.1, 'nhead': 4, 'attn_type': 'fac_dense', 'learning_rate': 1e-05, 'weight_decay': 1e-06}


In [6]:
Xtrain, Ytrain, Xtest, Ytest, XVal, YVal, scaler = make_dataset(df, target_col='log_returns',
                                                                exclude_cols=exclude_cols, timestep=timestep, ntest=ntest, horizon=horizon)
train_loader, val_loader, test_loader, test_loader_one = get_torch_data_loaders(
    Xtrain, Ytrain, Xtest, Ytest, XVal, YVal, batch_size
)

X.shape (1945, 5, 36) Y.shape (1945, 14)


In [13]:
from pytorch_lightning.callbacks import TQDMProgressBar

model = TransAm.load_from_checkpoint(checkpoint_path, map_location=torch.device('cpu'), loss_fn=RMSELoss)


early_stop_callback = EarlyStopping(monitor="val_loss", patience=patience, verbose=False, mode="min")
filename = f"{attn_type}_timestep={timestep}" + '{epoch}-{val_loss:.3f}'
checkpoint_callback = ModelCheckpoint(
    dirpath=f"checkpoints/{model_checkpoint_outputdir}",
    filename=filename,
    save_top_k=1,
    monitor="val_loss"
)


trainer = pl.Trainer(
    callbacks=[TQDMProgressBar(refresh_rate=0), early_stop_callback,checkpoint_callback],
    max_epochs=n_epochs,
    logger=True,
)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


kw_batch,feature size:  4 36
embed_dim: 36
num_heads:  4


In [12]:
trainer.fit(model, train_loader, val_loader)


  | Name                | Type                    | Params
----------------------------------------------------------------
0 | pos_encoder         | PositionalEncoding      | 0     
1 | encoder_layer       | TransformerEncoderLayer | 155 K 
2 | transformer_encoder | TransformerEncoder      | 620 K 
3 | decoder1            | Linear                  | 2.5 K 
----------------------------------------------------------------
777 K     Trainable params
0         Non-trainable params
777 K     Total params
3.110     Total estimated model params size (MB)
C:\Users\Olexandr\miniconda3\envs\diploma\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning:

The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.

C:\Users\Olexandr\miniconda3\envs\diploma\lib\site

training_loss tensor(1.8950, grad_fn=<SqrtBackward0>)
training_loss tensor(1.8402, grad_fn=<SqrtBackward0>)
training_loss tensor(1.6900, grad_fn=<SqrtBackward0>)
training_loss tensor(1.4584, grad_fn=<SqrtBackward0>)
training_loss tensor(0.9509, grad_fn=<SqrtBackward0>)
training_loss tensor(0.7283, grad_fn=<SqrtBackward0>)
training_loss tensor(0.7362, grad_fn=<SqrtBackward0>)
training_loss tensor(0.7093, grad_fn=<SqrtBackward0>)
training_loss tensor(0.8297, grad_fn=<SqrtBackward0>)
training_loss tensor(1.0805, grad_fn=<SqrtBackward0>)
training_loss tensor(1.0539, grad_fn=<SqrtBackward0>)
training_loss tensor(0.9810, grad_fn=<SqrtBackward0>)
training_loss tensor(0.9173, grad_fn=<SqrtBackward0>)
training_loss tensor(1.0095, grad_fn=<SqrtBackward0>)
training_loss tensor(1.1601, grad_fn=<SqrtBackward0>)
training_loss tensor(1.1499, grad_fn=<SqrtBackward0>)
training_loss tensor(1.0915, grad_fn=<SqrtBackward0>)
training_loss tensor(1.0926, grad_fn=<SqrtBackward0>)
training_loss tensor(1.1170,

C:\Users\Olexandr\miniconda3\envs\diploma\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning:

Detected KeyboardInterrupt, attempting graceful shutdown...

